In [ ]:
!pip install tensorflow==2.4.1
!pip install keras==2.4.3

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Reshape, Permute, Activation, Input, \
    add, multiply
from keras.layers import concatenate, core, Dropout
from keras.models import Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.core import Lambda
#import keras.backend as K

#%tensorflow_version 1.x
import os
import keras
from keras.callbacks import TensorBoard
import tensorflow as tf
#import keras.backend.tensorflow_backend as K
import keras.backend as K
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger


2021-09-16 09:36:48.688263: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import cv2
import warnings

warnings.filterwarnings("ignore")

BackGround = [255, 255, 255]
road = [0, 0, 0]
# COLOR_DICT = np.array([BackGround, road])
one = [128, 128, 128]
two = [128, 0, 0]
three = [192, 192, 128]
four = [255, 69, 0]
five = [128, 64, 128]
six = [60, 40, 222]
seven = [128, 128, 0]
eight = [192, 128, 128]
nine = [64, 64, 128]
ten = [64, 0, 128]
eleven = [64, 64, 0]
twelve = [0, 128, 192]
COLOR_DICT = np.array([one, two,three,four,five,six,seven,eight,nine,ten,eleven,twelve])


class data_preprocess:
    def __init__(self, train_path=None, image_folder=None, label_folder=None,
                 valid_path=None,valid_image_folder =None,valid_label_folder = None,
                 test_path=None, save_path=None,
                 img_rows=256, img_cols=256,
                 flag_multi_class=False,
                 num_classes = 2):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.train_path = train_path
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.valid_path = valid_path
        self.valid_image_folder = valid_image_folder
        self.valid_label_folder = valid_label_folder
        self.test_path = test_path
        self.save_path = save_path
        self.data_gen_args = dict(rotation_range=20,
                                  width_shift_range=0.002,
                                  shear_range=0.03,
                                  zoom_range=0.005,
                                  vertical_flip=True,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
        self.image_color_mode = "rgb"
        self.label_color_mode = "grayscale"

        self.flag_multi_class = flag_multi_class
        self.num_class = num_classes
        self.target_size = (256, 256)
        self.img_type = 'png'

    def adjustData(self, img, label):
        if (self.flag_multi_class):
            img = img / 255.
            label = label[:, :, :, 0] if (len(label.shape) == 4) else label[:, :, 0]
            new_label = np.zeros(label.shape + (self.num_class,))
            for i in range(self.num_class):
                new_label[label == i, i] = 1
            label = new_label
        elif (np.max(img) > 1):
            #img = img / 255.
            #label = label / 255.
            #label[label >= 0.5] = 1
            #label[label < 0.5] = 0
            img2 =np.asarray(img)
            label2 =np.asarray(label)
            img2 =img2.astype('float32')
            label2 =label2.astype('float32')
            img2 /= 255.0
            label2 /= 255.0
            label2[label2 >= 0.5] = 1
            label2[label2 < 0.5] = 0
        return (img2, label2)

    def trainGenerator(self, batch_size, image_save_prefix="image", label_save_prefix="label",
                       save_to_dir=None, seed=7):
        '''
        can generate image and label at the same time
        use the same seed for image_datagen and label_datagen to ensure the transformation for image and label is the same
        if you want to visualize the results of generator, set save_to_dir = "your path"
        '''
        image_datagen = ImageDataGenerator(**self.data_gen_args)
        label_datagen = ImageDataGenerator(**self.data_gen_args)
        image_generator = image_datagen.flow_from_directory(
            self.train_path,
            classes=[self.image_folder],
            class_mode=None,
            color_mode=self.image_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            save_to_dir=save_to_dir,
            save_prefix=image_save_prefix,
            seed=seed)
        label_generator = label_datagen.flow_from_directory(
            self.train_path,
            classes=[self.label_folder],
            class_mode=None,
            color_mode=self.label_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            save_to_dir=save_to_dir,
            save_prefix=label_save_prefix,
            seed=seed)
        train_generator = zip(image_generator, label_generator)
        for (img, label) in train_generator:
            img, label = self.adjustData(img, label)
            yield (img, label)

    def testGenerator(self):
        filenames = os.listdir(self.test_path)
        for filename in filenames:
            img = io.imread(os.path.join(self.test_path, filename), as_gray=False)
            img = img / 255.
            img = trans.resize(img, self.target_size, mode='constant')
            img = np.reshape(img, img.shape + (1,)) if (not self.flag_multi_class) else img
            img = np.reshape(img, (1,) + img.shape)
            yield img

    def validLoad(self, batch_size,seed=7):
        image_datagen = ImageDataGenerator(rescale=0)
        label_datagen = ImageDataGenerator(rescale=0)
        image_generator = image_datagen.flow_from_directory(
            self.valid_path,
            classes=[self.valid_image_folder],
            class_mode=None,
            color_mode=self.image_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            seed=seed)
        label_generator = label_datagen.flow_from_directory(
            self.valid_path,
            classes=[self.valid_label_folder],
            class_mode=None,
            color_mode=self.label_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            seed=seed)
        train_generator = zip(image_generator, label_generator)
        for (img, label) in train_generator:
            img, label = self.adjustData(img, label)
            yield (img, label)
        # return imgs,labels

    def saveResult(self, npyfile, size, name,threshold=80):
        for i, item in enumerate(npyfile):
            img = item
            img_std = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
            if self.flag_multi_class:
                for row in range(len(img)):
                    for col in range(len(img[row])):
                        num = np.argmax(img[row][col])
                        img_std[row][col] = COLOR_DICT[num]
            else:
                for k in range(len(img)):
                    for j in range(len(img[k])):
                        num = img[k][j]
                        if num < (threshold/255.0):
                            img_std[k][j] = road
                        else:
                            img_std[k][j] = BackGround
            img_std = cv2.resize(img_std, size, interpolation=cv2.INTER_CUBIC)
            cv2.imwrite(os.path.join(self.save_path, ("%s_predict." + self.img_type) % (name)), img_std)

In [ ]:
####  Metrics

from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf
import numpy as np
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss
def iou_coeff(y_true, y_pred):
    smooth=1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union=K.sum(y_true_f) + K.sum(y_pred_f)-intersection
    mvalue=(intersection+smooth)/(union+smooth)
    return mvalue
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def ACL5(y_true, y_pred): 

	#y_pred = K.cast(y_pred, dtype = 'float64')

	print(K.int_shape(y_pred))

	x = y_pred[:,1:,:,:] - y_pred[:,:-1,:,:] # horizontal and vertical directions 
	y = y_pred[:,:,1:,:] - y_pred[:,:,:-1,:]

	delta_x = x[:,1:,:-2,:]**2
	delta_y = y[:,:-2,1:,:]**2
	delta_u = K.abs(delta_x + delta_y) 

	epsilon = 0.00000001 # where is a parameter to avoid square root is zero in practice.
	w = 1####
	lenth = w * K.sum(K.sqrt(delta_u + epsilon)) # equ.(11) in the paper


	C_1 = np.ones((256, 256))
	C_2 = np.zeros((256, 256))

	region_in = K.abs(K.sum( y_pred[:,:,:,0] * ((y_true[:,:,:,0] - C_1)**2) ) ) # equ.(12) in the paper
	region_out = K.abs(K.sum( (1-y_pred[:,:,:,0]) * ((y_true[:,:,:,0] - C_2)**2) )) # equ.(12) in the paper

	lambdaP = 5 # lambda parameter could be various.
	
	loss =  lenth + lambdaP * ((region_in) + (region_out)) 

	return loss
def ACL5_mod(y_true, y_pred): 

	#y_pred = K.cast(y_pred, dtype = 'float64')

	print(K.int_shape(y_pred))

	x = y_pred[:,1:,:,:] - y_pred[:,:-1,:,:] # horizontal and vertical directions 
	y = y_pred[:,:,1:,:] - y_pred[:,:,:-1,:]

	delta_x = x[:,1:,:-2,:]**2
	delta_y = y[:,:-2,1:,:]**2
	delta_u = K.abs(delta_x + delta_y) 

	epsilon = 0.00000001 # where is a parameter to avoid square root is zero in practice.
	w = 1####
	lenth = w * K.sum(K.sqrt(delta_u + epsilon)) # equ.(11) in the paper


	C_1 = np.ones((256, 256))
	C_2 = np.zeros((256, 256))

	region_in = K.abs(K.sum( y_pred[:,:,:,0] * ((y_true[:,:,:,0] - C_1)**2) ) ) # equ.(12) in the paper
	region_out = K.abs(K.sum( (1-y_pred[:,:,:,0]) * ((y_true[:,:,:,0] - C_2)**2) )) # equ.(12) in the paper

	lambdaP = 5 # lambda parameter could be various.
	
	loss =  lenth + lambdaP * ((region_in) + (region_out*1.4)) 

	return loss

In [ ]:
beta = 0.25
alpha = 0.25
gamma = 2
epsilon = 1e-5
smooth = 1

def tversky_index( y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (
                1 - alpha) * false_pos + smooth)

def tversky_loss( y_true, y_pred):
    return 1 - tversky_index(y_true, y_pred)

def focal_tversky( y_true, y_pred):
    pt_1 = tversky_index(y_true, y_pred)
    gamma = 0.75
    return K.pow((1 - pt_1), gamma)

def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [ ]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [ ]:
!pip install segmentation_models
import segmentation_models
from segmentation_models.losses import bce_jaccard_loss

     |████████████████████████████████| 50 kB 5.3 MB/s  eta 0:00:01
Segmentation Models: using `tf.keras` framework.


In [ ]:
def ACL5_bce_jaccard_loss(y_true, y_pred):
    loss = ACL5(y_true, y_pred) + bce_jaccard_loss(y_true, y_pred)
    return loss


def focal_tversky_bce_jaccard_loss(y_true, y_pred):
    loss = focal_tversky(y_true, y_pred) + 2*bce_jaccard_loss(y_true, y_pred)
    return loss



In [ ]:

from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate
from tensorflow.keras.applications import DenseNet201

from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2

#  DenseNet201_UNet

In [ ]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate
from tensorflow.keras.applications import DenseNet201

from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2
def conv_block(inputs , num_filters):
  x = Conv2D(num_filters , 3 , padding= "same" )(inputs)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.3)(x)

  x = Conv2D(num_filters , 3 , padding= "same" )(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.3)(x)

  return x


def decoder_block(inputs , skip_features , num_filters):

  x = Conv2DTranspose(num_filters , (2, 2) , strides = 2 , padding = "same")(inputs)
  x = Concatenate()([x , skip_features ])
  x = conv_block(x , num_filters)

  return x


from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50


def build_DenseNet210_Unet(input_shape):

  ##Input
  inputs = Input(input_shape)

  ## Pre-Trained Encoder

  encoder = DenseNet201(include_top = False , weights = "imagenet" , input_tensor = inputs)

  # Skip Connections which are the Features and we will access these Features and Feed it in the Decoder
  #First Feature is the input shaped image
  s1 = encoder.get_layer("input_1").output                         # 256 * 256

  s2 = encoder.get_layer("conv1/relu").output                       # 128 * 128 

  s3 = encoder.get_layer("pool2_relu").output       # 64 * 64 

  s4 = encoder.get_layer("pool3_relu").output       # 32 * 32

  ## BottleNeck or bridge 
  b1 = encoder.get_layer("pool4_relu").output       # 16 * 16

  ## Decoder
  d1 = decoder_block(b1 , s4 , 512)                                # 32 * 32 * 512 where 512 is the number of Features we extracted 

  d2 = decoder_block(d1 , s3 , 256)                                # 64 * 64

  d3 = decoder_block(d2 , s2 , 128)                                # 128 * 128

  d4 = decoder_block(d3 , s1 , 64)                                # 128 * 128

  ## OUTPUT 
  x = Conv2D( 2 ,  1  ,  padding = "same" ,  activation = "relu")(d4)
  outputs = Conv2D( 1 ,  1  ,  padding = "same" ,  activation = "sigmoid")(x)

  ## MODEL 

  model = Model(inputs , outputs ,  name = "DenseNet210")

  return model
  #encoder.summary()

In [ ]:
lrate = 7.00E-05 

    
if __name__ == "__main__":
    input_shape = (256, 256, 3)
    model = build_DenseNet210_Unet(input_shape)
    model.summary()
    model_checkpoint1 = keras.callbacks.ModelCheckpoint('att_r2_unet.hdf5', monitor='val_dice_loss',verbose=1,mode='min',save_best_only=True)
    csv_logger = CSVLogger('trainingRes2Net.log', append=True, separator=';')
    model.compile(optimizer=Adam(lr=lrate), loss=ACL5 , metrics=[ACL5 ,bce_jaccard_loss , dice_coef , dsc,  dice_loss,iou_coeff,precision,recall])

2021-09-16 09:37:01.061922: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-16 09:37:01.065844: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-16 09:37:01.118692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-16 09:37:01.119669: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-16 09:37:01.119749: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-16 09:37:01.145670: I tensorflow/stream_executor/platform/def

74842112/74836368 [==============================] - 1s 0us/step
Model: "DenseNet210"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
_______________________

In [ ]:
   
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


#path to images
train_path = "../input/training/training"
image_folder = "images"
label_folder = "label"
valid_path =  "../input/validation/Validation"
valid_image_folder ="images"
valid_label_folder = "label"
log_filepath = './log'
flag_multi_class = False
num_classes = 2
dp = data_preprocess(train_path=train_path,image_folder=image_folder,label_folder=label_folder,
                     valid_path=valid_path,valid_image_folder=valid_image_folder,valid_label_folder=valid_label_folder,
                     flag_multi_class=flag_multi_class,
                     num_classes=num_classes)

train_data = dp.trainGenerator(batch_size=2)
valid_data = dp.validLoad(batch_size=1)
test_data = dp.testGenerator()
lrate = 7.00E-05 


from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D

In [ ]:
history = model.fit_generator(train_data,
                              steps_per_epoch=1912,epochs=40,
                              validation_steps=207,
                              validation_data=valid_data,
                              callbacks=[model_checkpoint1,csv_logger])

Found 600 images belonging to 1 classes.
Found 600 images belonging to 1 classes.


2021-09-16 09:37:08.790458: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-16 09:37:08.794516: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000185000 Hz


Epoch 1/40
(None, 256, 256, 1)
(None, 256, 256, 1)
(None, 256, 256, 1)
(None, 256, 256, 1)


2021-09-16 09:37:22.870081: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-09-16 09:37:28.010056: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-16 09:37:28.707462: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


1912/1912 [==============================] - ETA: 0s - loss: 287033.6292 - ACL5: 287033.6292 - binary_crossentropy_plus_jaccard_loss: 1.5673 - dice_coef: 0.0283 - dsc: 0.0284 - dice_loss: 0.9716 - iou_coeff: 0.0144 - precision: 0.0172 - recall: 0.0385Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
(None, 256, 256, 1)
(None, 256, 256, 1)
1912/1912 [==============================] - 259s 124ms/step - loss: 287007.4841 - ACL5: 287007.4841 - binary_crossentropy_plus_jaccard_loss: 1.5672 - dice_coef: 0.0283 - dsc: 0.0284 - dice_loss: 0.9716 - iou_coeff: 0.0144 - precision: 0.0172 - recall: 0.0385 - val_loss: 69509.2578 - val_ACL5: 69509.2578 - val_binary_crossentropy_plus_jaccard_loss: 1.2379 - val_dice_coef: 0.0484 - val_dsc: 0.0484 - val_dice_loss: 0.9516 - val_iou_coeff: 0.0250 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00

Epoch 00001: val_dice_loss improved from inf to 0.95161, saving model to att_r2_unet.hdf5
Epoch 2/40
1912/1912 [=============